El objetivo de este notebook es plotear y analizar tiempos / spikes de experimentos. Por un lado cargamos un archivo donde se indica el tiempo de activación del láser, y por otro tenemos los spikes. Tenemos que plotear primero, de manera coordinada, la información de ambos archivos.

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib as mpl
import numpy as np
import time
import datetime as dt
import os
import seaborn as sns


In [ ]:
hoy = dt.date.today()

In [ ]:
df_times = pd.read_csv('tiempos.csv', sep=', ', index_col='FileName')
df_times

In [ ]:
exp_date = '2024_02_20'

name_sample = 'D19_POSTsiembra_C2_(IV)(000)'
name_file_spikes = f'{name_sample}_spike_list.csv'
name_file_laser = '202402020_MEAs_C2_(IV).csv'

column_time = 'AddedDate'

timestamp = [int(i) for i in df_times[column_time].loc[name_sample].split(':')]

time_absolute_start = dt.datetime.combine(hoy, dt.time(timestamp[0], timestamp[1], timestamp[2], timestamp[3] * 1000))  # A1_(IV)_(000)

time_window_seconds = 0.5         # We add a custom window to displace the times of the lasers, because we see that in some cases the laser start time is not correct

os.makedirs(f'../../results/{exp_date}/{column_time}', exist_ok=True)

In [ ]:
df_spikes = pd.read_csv(f"../../spike_lists/{exp_date}/{name_file_spikes}", sep=None, encoding='unicode_escape') #Use sep=None to recognize the separator automatically, because in some files it is ; and in other it is \t
df_spikes['electrode_str'] = [i.split('_')[1] for i in df_spikes['Electrode']]
df_spikes['electrode_coord'] = [(int(electrode[0]) - 1, 3 - (int(electrode[1]) - 1)) for electrode in df_spikes['electrode_str']]
df_spikes

In [ ]:
df_laser = pd.read_csv(f"../../tiempos_structura/{exp_date}/{name_file_laser}", index_col=0)

list_start_times, list_end_times = [], []


# Modify dates
for value_on, value_off in zip(df_laser['Turn ON Laser Time'].values, df_laser['Turn OFF Laser Time'].values):
    value_on, value_off = value_on.replace(':', '.'), value_off.replace(':', '.')
    tuple_on = (int(value_on.split('.')[0]), int(value_on.split('.')[1]), int(value_on.split('.')[2]), int(value_on.split('.')[3]))
    tuple_off = (int(value_off.split('.')[0]), int(value_off.split('.')[1]), int(value_off.split('.')[2]), int(value_off.split('.')[3]))

    list_start_times.append((dt.datetime.combine(hoy, dt.time(tuple_on[0], tuple_on[1], tuple_on[2], tuple_on[3]))  - time_absolute_start).total_seconds() + time_window_seconds)
    list_end_times.append((dt.datetime.combine(hoy, dt.time(tuple_off[0], tuple_off[1], tuple_off[2], tuple_off[3]))  - time_absolute_start).total_seconds() + time_window_seconds)


df_laser['start_times'], df_laser['end_times'] = list_start_times, list_end_times
df_laser['duration'] = df_laser['ON duration (ms)'] / 1000
df_laser



In [ ]:
voltages = sorted(set(df_laser['Laser Power (mW)'].values))
electrodes = sorted(set(df_spikes['electrode_str'].values))

window = 2

fig, axs = plt.subplots(len(voltages), 1, figsize=(18, 25))


for idx, voltage in enumerate(voltages):
    df_laser_sub = df_laser[df_laser['Laser Power (mW)'] == voltage]
    t_0, t_f = min(df_laser_sub['start_times']), max(df_laser_sub['end_times'])


    for rep, start, end in zip(df_laser_sub['Repetition'].values, df_laser_sub['start_times'].values, df_laser_sub['end_times'].values):
        axs[idx].add_patch(mpl.patches.Rectangle((start, -1), end - start, len(electrodes)+1, alpha=0.25, color='#800000'))

    for electrode_idx, electrode in enumerate(electrodes):
        df_spikes_sub = df_spikes[(df_spikes['Time (s)'] > t_0 - window) & (df_spikes['Time (s)'] < t_f + window) & (df_spikes['electrode_str'] == electrode)]
        axs[idx].scatter(x=df_spikes_sub['Time (s)'].values, y=[electrode_idx] * len(df_spikes_sub), marker='|', s=100, c='#7f7f7f')
    


    axs[idx].set_yticks(np.array(range(len(electrodes))))
    axs[idx].set_yticklabels(electrodes)
    axs[idx].set_ylim([-1, len(electrodes)])
    axs[idx].set_xlim([t_0 - window, t_f+window])
    axs[idx].tick_params(left = False)
    axs[idx].set_xlabel('Time (s)')
    axs[idx].set_ylabel('Electrodes')

    axs[0].title.set_text('0 mW')
    axs[1].title.set_text('20 mW')
    axs[2].title.set_text('40 mW')
    axs[3].title.set_text('60 mW')
    axs[4].title.set_text('80 mW')
    axs[5].title.set_text('100 mW')

plt.subplots_adjust(left=0.1,
                    bottom=0.1,
                    right=0.9,
                    top=0.9,
                    wspace=0.4,
                    hspace=0.4)


plt.savefig(f'../../results/{exp_date}/{column_time}/{name_sample}_spikeplot.png', dpi=300)

# Calculating the mean representation time between pulses

In [ ]:
tw = 0.1
cutoff_counts = 60

time_ranges = np.arange(np.min(df_laser['start_times']) , np.max(df_laser['end_times']) , tw)
time_ranges_mean = (time_ranges[1:] + time_ranges[:-1]) * 0.5 

counts = np.array([len(df_spikes[(df_spikes['Time (s)'] >= start) & (df_spikes['Time (s)'] < end)]) for start, end in zip(time_ranges[:-1], time_ranges[1: ])])
len(counts)


In [ ]:
fig = plt.figure(figsize=(3,5))
tmps = np.argwhere(counts > cutoff_counts).flatten() * tw
diff = tmps[1: ] - tmps[: -1]
sns.violinplot(diff[diff > 3 * tw])
plt.xlabel(name_sample)
plt.ylabel('Time between bursts')
plt.tight_layout()

plt.savefig(f'../../results/{exp_date}/{column_time}/{name_sample}_distribution_interspike_gap_duration.png', dpi=300)

In [ ]:
df_spikes

# Calculating the mean firing rate

In [ ]:
list_voltages_MFR, list_electrodes_MFR, list_MFR = [], [], []


for idx_vol, voltage in enumerate(voltages):
    for idx_elec, electrode in enumerate(electrodes):
        df_laser_sub = df_laser[df_laser['Laser Power (mW)'] == voltage]
        t_0, t_f = min(df_laser_sub['start_times']), max(df_laser_sub['end_times'])
        

        df_spikes_sub = df_spikes[(df_spikes['electrode_str'] == electrode) & (df_spikes['Time (s)'] > t_0) & (df_spikes['Time (s)'] < t_f)]
        MFR = len(df_spikes_sub) / (t_f - t_0)
        list_MFR.append(MFR)
        list_voltages_MFR.append(voltage)
        list_electrodes_MFR.append(electrode)

df_MFR = pd.DataFrame({'voltage': list_voltages_MFR, 'electrode': list_electrodes_MFR, 'MFR': list_MFR})

df_MFR.to_csv(f'../../results/{exp_date}/{column_time}/{name_sample}_MRF_raw.csv', sep=';', index=None)
df_MFR

In [ ]:
df_MRF_pivot = df_MFR.pivot(index='electrode', columns='voltage', values='MFR')

df_MRF_pivot.to_csv(f'../../results/{exp_date}/{column_time}/{name_sample}_MRF_pivot.csv', sep=';')
df_MRF_pivot

In [ ]:
df_MRF_pivot_norm = df_MRF_pivot.copy()
for voltage in voltages[1:]:
    df_MRF_pivot_norm[voltage] /= df_MRF_pivot_norm[voltages[0]]

df_MRF_pivot_norm = df_MRF_pivot_norm.iloc[:, 1:] * 100

df_MRF_pivot_norm.to_csv(f'../../results/{exp_date}/{column_time}/{name_sample}_MRF_pivot_norm.csv', sep=';')
df_MRF_pivot_norm

In [ ]:
df_MFR_summary = pd.DataFrame({'voltage': voltages, 
                               'sum': df_MFR.groupby('voltage').sum()['MFR'].values, 
                               'mean': df_MFR.groupby('voltage').mean()['MFR'].values, 
                               'median': df_MFR.groupby('voltage').median()['MFR'].values, 
                               'std': df_MFR.groupby('voltage').std()['MFR'].values})

df_MFR_summary.to_csv(f'../../results/{exp_date}/{column_time}/{name_sample}_MRF_summary.csv', sep=';', index=None)
df_MFR_summary